## Limpeza de dados Cartola ano 2014

In [ ]:
from pyspark.sql import HiveContext
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lower, col, lit, regexp_replace
import pandas as pd
import json
import requests

## Times

In [ ]:
times_2014_df = spark.read.csv("/cartola/data/2014/2014_times.csv", header=True)
sorted_times_2014_df = times_2014_df.sort(times_2014_df.ID.asc())

#Chegar se o arquivo está no hdfs

times_2014_ano = times_2014_df.withColumn('ano', lit(2014))
times_2014_ano.toPandas()

In [ ]:
times_2014_ano.write.parquet('/cartola/clean/times/2014')
times_2014_parquet = spark.read.parquet('/cartola/clean/times/*')

In [ ]:
times_2014_parquet.toPandas()
times_2014_parquet.schema

#Alterar ID para int.(está string)

In [ ]:
times_2014_parquet = times_2014_parquet.withColumn("ID", times_2014_parquet["ID"].cast(IntegerType()))

In [ ]:
times_2014_parquet.schema

In [ ]:
times_2014_parquet.toPandas()

## Jogadores

In [ ]:
#Carrega arquivo CSV
jogadores_2014_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2014/2014_jogadores.csv", header=True)
sorted_jogadores_2014_df = jogadores_2014_df.sort(jogadores_2014_df.ID.asc())


In [ ]:
#Adiciona coluna ANO = 2014
jogadores_2014_ano = sorted_jogadores_2014_df.withColumn('ano', lit(2014))
jogadores_2014_ano.toPandas()

In [ ]:
jogadores_2014_ano.write.parquet('/cartola/clean/jogadores/2014')

In [ ]:
#transforma arquivo em parquet
jogadores_2014_parquet = spark.read.parquet('/cartola/clean/jogadores/*')

In [ ]:
jogadores_2014_parquet.show()
jogadores_2014_parquet.schema


In [ ]:
#Alterar ID para int.(está string)
jogadores_2014_parquet = jogadores_2014_parquet.withColumn("ID", jogadores_2014_parquet["ID"].cast(IntegerType()))
jogadores_2014_parquet = jogadores_2014_parquet.withColumn("ClubeID", jogadores_2014_parquet["ClubeID"].cast(IntegerType()))
jogadores_2014_parquet = jogadores_2014_parquet.withColumn("PosicaoID", jogadores_2014_parquet["PosicaoID"].cast(IntegerType()))

In [ ]:
jogadores_2014_parquet.schema

In [ ]:
jogadores_2014_parquet.show()

print(jogadores_2014_parquet.count())

## Partidas_IDS

In [ ]:
partidas_ids_2014_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2014/2014_partidas_ids.csv", header=True)

In [ ]:
partidas_ids_2014_ano = partidas_ids_2014_df.withColumn('ano', lit(2014))

In [ ]:
partidas_ids_2014_ano.show(100)

In [ ]:
partidas_ids_2014_ano = partidas_ids_2014_ano.withColumnRenamed("Casa", "CasaID")
partidas_ids_2014_ano = partidas_ids_2014_ano.withColumnRenamed("Visitante", "VisitanteID")

In [ ]:
partidas_ids_2014_ano.write.parquet('/cartola/clean/partidas_ids/2014')

In [ ]:
partidas_ids_2014_parquet = spark.read.parquet('/cartola/clean/partidas_ids/*')

In [ ]:
partidas_ids_2014_parquet.show(100)
partidas_ids_2014_parquet.schema

In [ ]:
partidas_ids_2014_parquet = partidas_ids_2014_parquet.withColumn("ID", partidas_ids_2014_parquet["ID"].cast(IntegerType()))
partidas_ids_2014_parquet = partidas_ids_2014_parquet.withColumn("Rodada", partidas_ids_2014_parquet["Rodada"].cast(IntegerType()))
partidas_ids_2014_parquet = partidas_ids_2014_parquet.withColumn("CasaID", partidas_ids_2014_parquet["CasaID"].cast(IntegerType()))
partidas_ids_2014_parquet = partidas_ids_2014_parquet.withColumn("VisitanteID", partidas_ids_2014_parquet["VisitanteID"].cast(IntegerType()))
partidas_ids_2014_parquet = partidas_ids_2014_parquet.withColumn("PlacarCasa", partidas_ids_2014_parquet["PlacarVisitante"].cast(IntegerType()))
partidas_ids_2014_parquet = partidas_ids_2014_parquet.withColumn("PlacarVisitante", partidas_ids_2014_parquet["PlacarVisitante"].cast(IntegerType()))

In [ ]:
partidas_ids_2014_parquet.show()

print(partidas_ids_2014_parquet.count())

In [ ]:
inner_join = partidas_ids_2014_ano.join(times_2014_ano, partidas_ids_2014_ano.Casa == times_2014_ano.ID)
inner_join.toPandas()

## LANCES (Criar a tabela de scouts a partir da tabela lances)

In [ ]:
lances_2014_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2014/2014_lances.csv", header=True)

In [ ]:
lances_2014_ano = lances_2014_df.withColumn('ano', lit(2014))

In [ ]:
lances_2014_ano.show(1000)

In [ ]:
lances_2014_ano.write.parquet('/cartola/clean/lances/2014/')

In [ ]:
lances_2014_parquet = spark.read.parquet('/cartola/clean/lances/*')

In [ ]:
lances_2014_parquet.show()
lances_2014_parquet.schema

In [ ]:
lances_2014_parquet = lances_2014_parquet.withColumn("ID", lances_2014_parquet["ID"].cast(IntegerType()))
lances_2014_parquet = lances_2014_parquet.withColumn("PartidaID", lances_2014_parquet["PartidaID"].cast(IntegerType()))
lances_2014_parquet = lances_2014_parquet.withColumn("ClubeID", lances_2014_parquet["ClubeID"].cast(IntegerType()))
lances_2014_parquet = lances_2014_parquet.withColumn("AtletaID", lances_2014_parquet["AtletaID"].cast(IntegerType()))
lances_2014_parquet = lances_2014_parquet.withColumn("Momento", lances_2014_parquet["Momento"].cast(IntegerType()))

In [ ]:
lances_2014_parquet.show()

print(lances_2014_parquet.count())

## scouts_raw

In [ ]:
scouts_raw_2014_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2014/2014_scouts_raw.csv", header=True)

In [ ]:
scouts_raw_2014_ano = scouts_raw_2014_df.withColumn('ano', lit(2014))

In [ ]:
scouts_raw_2014_ano.show()

In [ ]:
# Removendo colunas
scouts_raw_2014_ano = scouts_raw_2014_ano.drop('PrecoVariacao', 'Mando', 'Titular', 'Substituido', 'TempoJogado', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS', 'Participou', 'Posicao', 'Partida', 'Nota', 'Jogos')

In [ ]:
# remove todas as linhas com rodada == 0
scouts_raw_2014_ano_clean = scouts_raw_2014_ano[scouts_raw_2014_ano['Rodada'] > 0]

In [ ]:
# remove todos os jogadores que não participaram de alguma rodada
#scouts_raw_2014_ano_clean = scouts_raw_2014_ano_clean[(scouts_raw_2014_ano_clean['Participou'] == True) | (scouts_raw_2014_ano_clean['PrecoVariacao'] != 0)]

In [ ]:
scouts_raw_2014_ano_clean.show()

In [ ]:
#ID|PartidaID|ClubeID|AtletaID|Periodo|Momento|Tipo| ano
scouts_raw_2014_ano_clean = scouts_raw_2014_ano_clean.withColumnRenamed("Atleta", "AtletaID")
scouts_raw_2014_ano_clean = scouts_raw_2014_ano_clean.withColumnRenamed("Clube", "ClubeID")

In [ ]:
scouts_raw_2014_ano_clean.write.parquet('/cartola/clean/scouts_raw/2014')

In [ ]:
scouts_raw_2014_parquet = spark.read.parquet('/cartola/clean/scouts_raw/*')

In [ ]:
#scouts_raw_2014_parquet.show(10000)
scouts_raw_2014_parquet.limit(100).toPandas()

In [ ]:
scouts_raw_2014_parquet.schema

In [ ]:
#deve-se converter valores em BigDecimal?
#Retirar atleta sem clube
scouts_raw_2014_parquet = scouts_raw_2014_parquet.withColumn("AtletaID", scouts_raw_2014_parquet["AtletaID"].cast(IntegerType()))
scouts_raw_2014_parquet = scouts_raw_2014_parquet.withColumn("Rodada", scouts_raw_2014_parquet["Rodada"].cast(IntegerType()))
scouts_raw_2014_parquet = scouts_raw_2014_parquet.withColumn("ClubeID", scouts_raw_2014_parquet["ClubeID"].cast(IntegerType()))
scouts_raw_2014_parquet = scouts_raw_2014_parquet.withColumn("Jogos", scouts_raw_2014_parquet["Jogos"].cast(IntegerType()))

In [ ]:
#scouts_raw_2014_parquet.show()
scouts_raw_2014_parquet.toPandas()
#print(scouts_raw_2014_parquet.count())

## DEMONSTRAÇÃO

In [ ]:
partidas_2014_df = spark.read.csv("/cartola/data/2014/2014_jogadores.csv", header=True)
partidas_2014_ct = partidas_2014_df.withColumn('time', regexp_replace('home_team', ' - RJ', ''))
final_partidas = partidas_2014_ct.withColumn('time_low', lower(col('time'))).show(truncate=False)

In [ ]:
with_ano_partidas = partidas_2014_ct.withColumn('ano', lit(2014)).show(truncate=False)
with_ano_partidas.show()

In [ ]:
Comando para sobrescrever arquivo caso já existente.
Agrupar scouts por ID e contar (caso tenha duplicidade)

In [ ]:
inner_join = partidas_ids_2014_ano.join(times_2014_ano, partidas_ids_2014_ano.Casa == times_2014_ano.ID)
inner_join.toPandas()